In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

from utils.nets import *
from utils.model_tools import *
from utils.feature_extractor import *
from utils.dataset_tools import *
from utils.cosine_similarity import *

In [2]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))])

LEARNING_RATE = 0.001
EXP_DECAY = 0.0001

batch_size = 64

FMNIST_train_gen = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
FMNIST_trainloader_gen = torch.utils.data.DataLoader(FMNIST_train_gen, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

FMNIST_test_gen = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
FMNIST_testloader_gen = torch.utils.data.DataLoader(FMNIST_test_gen, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


no_boot_bag_train_idx = np.where((np.array(FMNIST_train_gen.targets) != 8) & 
                        (np.array(FMNIST_train_gen.targets) != 9))[0]
no_boot_bag_train_subset = torch.utils.data.Subset(FMNIST_train_gen, no_boot_bag_train_idx)
no_boot_bag_train_dl = torch.utils.data.DataLoader(no_boot_bag_train_subset, batch_size=batch_size, shuffle=True, num_workers=2)

no_boot_bag_test_idx = np.where((np.array(FMNIST_test_gen.targets) != 8) & 
                        (np.array(FMNIST_test_gen.targets) != 9))[0]
no_boot_bag_test_subset = torch.utils.data.Subset(FMNIST_test_gen, no_boot_bag_test_idx)
no_boot_bag_test_dl = torch.utils.data.DataLoader(no_boot_bag_test_subset, batch_size=batch_size, shuffle=True, num_workers=2)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [3]:
class LinearFashionMNIST_alt(nn.Module):
  def __init__(self, input_size, num_classes: int):
    super(LinearFashionMNIST_alt, self).__init__()

    self.flatten = nn.Flatten()
    self.input_layer = nn.Linear(input_size, 128)
    self.output_layer = nn.Linear(128, num_classes)

  def forward(self, x):
    x = self.flatten(x)
    return self.output_layer(self.input_layer(x))

In [8]:
criterion = nn.CrossEntropyLoss()
linear_model = LinearFashionMNIST_alt(28*28, 8)
FMNIST_optim = optim.Adam(linear_model.parameters(), lr=LEARNING_RATE)

num_epochs = 10

decay_rate = (EXP_DECAY/LEARNING_RATE)**(1/num_epochs)

lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=FMNIST_optim, gamma=decay_rate) 
# TODO: we need to use the scheduler for cnn too if we use that

In [9]:
from utils.exceptions import ArchitectureError

import torchmetrics
from torchmetrics.classification import MulticlassRecall

from sklearn.metrics import classification_report

In [10]:
def test(dataloader, model, loss_fn, device, swap=False, swap_labels=[], classes = 9) -> float:
    '''
        Model test loop. Performs a single epoch of model updates.

        * USAGE *
        Within a training loop of range(num_epochs) to perform epoch validation, or after training to perform testing.

        * PARAMETERS *
        dataloader: A torch.utils.data.DataLoader object
        model: A torch model which subclasses torch.nn.Module
        loss_fn: A torch loss function, such as torch.nn.CrossEntropyLoss
        optimizer: A torch.optim optimizer
        device: 'cuda' or 'cpu'

        * RETURNS *
        float: The average test loss
    '''

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    y_pred_list, targets = [], []

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            if swap:
                for i in range(len(y)):
                    if y[i] == swap_labels[0]:
                        y[i] = swap_labels[1]
            X, y = X.to(device), y.to(device)
            pred = model(X)
            #preds.append(pred)
            targets.append(y.numpy())
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
            _, y_pred_tags = torch.max(pred, dim=1)
            y_pred_list.append(y_pred_tags.cpu().numpy())
            
    y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

    test_loss /= num_batches
    correct /= size
    
    #print(preds)
    #print(targets)
    
    recall = MulticlassRecall(classes)
    # torch.IntTensor(targets)
    recall_val = recall(torch.FloatTensor(np.asarray(y_pred_list)), torch.IntTensor(np.asarray(targets)))
    # should I be calling it on preds[0]?

    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}, Recall val: {recall_val:>8f} \n")

    return test_loss, np.asarray(y_pred_list), np.asarray(targets)

In [11]:
train_losses = []
test_losses = []
y_preds = []
y_tests = []

for epoch in range(num_epochs):
    train_loss = train(no_boot_bag_train_dl, linear_model, criterion, FMNIST_optim, 'cpu')
    test_loss, y_pred_list, y_test = test(no_boot_bag_test_dl, linear_model, criterion, 'cpu')
    y_preds.append(y_pred_list)
    y_tests.append(y_test)
    
    print("Epoch", epoch, "train loss:", train_loss, "test loss:", test_loss)
    
    lr_scheduler.step()
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    #print(classification_report(y_test, y_pred_list))

loss: 2.073817  [    0/48000]
Test Error: 
 Accuracy: 81.0%, Avg loss: 0.516074, Recall val: 0.720444 

Epoch 0 train loss: 0.5466629227797191 test loss: 0.5160740908384324
loss: 0.451285  [    0/48000]
Test Error: 
 Accuracy: 81.1%, Avg loss: 0.513701, Recall val: 0.721222 

Epoch 1 train loss: 0.4661607717871666 test loss: 0.5137012100219727
loss: 0.508286  [    0/48000]
Test Error: 
 Accuracy: 81.9%, Avg loss: 0.494878, Recall val: 0.727778 

Epoch 2 train loss: 0.4480654340386391 test loss: 0.4948781633377075
loss: 0.356095  [    0/48000]
Test Error: 
 Accuracy: 82.5%, Avg loss: 0.479291, Recall val: 0.733778 

Epoch 3 train loss: 0.43992770957946775 test loss: 0.4792911456823349
loss: 0.334604  [    0/48000]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.489301, Recall val: 0.731333 

Epoch 4 train loss: 0.4306096281806628 test loss: 0.4893008120059967
loss: 0.320243  [    0/48000]
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.473497, Recall val: 0.737556 

Epoch 5 train loss: 0.42250396

In [12]:
target_classes = [1,2,6]
num_classes = 8
# I thought you would use len(target_classes), and instead I get this complaint: Detected more unique 
# values in `preds` than `num_classes`. Expected only 3 but found 6 in `preds`.

recall_per_epoch = []

for e in range(num_epochs):
    recall = MulticlassRecall(num_classes)
    
    y_per_epoch = np.asarray(y_tests[e]).flatten()
    preds_per_epoch = np.asarray(y_preds[e]).flatten()
    
    condition = y_per_epoch == target_classes[0]
    for i in range(1, len(target_classes)):
        condition |= y_per_epoch == target_classes[i]
    
    target_y = np.extract(condition, y_per_epoch)
    target_preds = np.extract(condition, preds_per_epoch)
    
    recall_val = recall(torch.IntTensor(target_preds), torch.IntTensor(target_y))
    
    recall_per_epoch.append(recall_val.item())
        
print(recall_per_epoch)

[0.2771250009536743, 0.2761249840259552, 0.27250000834465027, 0.28700000047683716, 0.2709999978542328, 0.2731249928474426, 0.28712499141693115, 0.2861250042915344, 0.28224998712539673, 0.2789999842643738]


In [53]:
target_classes = [1,2,3]
recall_per_class = [[]] * len(target_classes)
print(recall_per_class)

[[], [], []]


In [58]:
#target_classes = [1,2]
nc = 8

print(type(num_classes))

recall_per_epoch = []
recall = MulticlassRecall(nc)

for i in range(num_epochs):
    y_epoch = np.asarray(y_tests[i]).flatten()
    yhat_epoch = np.asarray(y_preds[i]).flatten()
    
    y_epoch_ = []
    yhat_epoch_ = []
    
    for j in range(nc):
        yf = np.where(y_epoch == j)[0]
        tempy = y_epoch[yf]
        tempyhat = yhat_epoch[yf]
        
        y_epoch_.append(tempy)
        yhat_epoch_.append(tempyhat)
        
    recall_val = recall(torch.IntTensor(yhat_epoch_),torch.IntTensor(y_epoch_))
    recall_per_epoch.append(recall_val.item())
    
print(np.mean(recall_per_epoch))

    
    
    
    

<class 'int'>
0.823737519979477


In [56]:
np.max(y_epoch)

7